In [1]:
!pip install transformers[ja,sentencepice]

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 27.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 26.8 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 30.3 MB/s eta 0:00:0000:0100:01
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556721 sha256=f974b58887cd58ee44fbe2bcbedfacb4bd30cac57464be1747b132309e9fb6b4
  Stored in directory: /tmp/pip-ephem-wheel-cache-7de1mjc8/wheels/5b/ea/e3/2f6e0860a327daba3b030853fce4483ed37468bbf1101c59c3
  Created wheel for unidic: filename=unidic-1.1.0-py3-none-any.whl size=7425 sha256=987daaeb51a323b904fd7e23f99c736d1a65db737c07289b73887db6daa7d91e
  Stored in directory: 

In [1]:
from transformers import pipeline

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 文章分類

In [7]:
text_classification_pipeline = pipeline(
    model = 'llm-book/bert-base-japanese-v3-marc_ja'
)
positive_text = '世界には言葉はわからなくても感動する音楽がある。'
# positive_textの極性を予測
print(text_classification_pipeline(positive_text)[0])

Downloading: 100%|██████████| 1.34k/1.34k [00:00<00:00, 10.2MB/s]


{'label': 'positive', 'score': 0.9993748068809509}


In [8]:
# negative_textの極性を予測
negative_text = '世界には言葉にできないほどひどい音楽がある。'
print(text_classification_pipeline(negative_text)[0])

{'label': 'negative', 'score': 0.9601457118988037}


### 自然言語推論

In [9]:
nli_pipeline = pipeline(model='llm-book/bert-base-japanese-v3-jnli')
text = '二人の男性がジェット機を見ています。'
entailment_text = 'ジェット機を見ている人が二人がいます。'

# textとentailment_textの論理関係を推測
print(nli_pipeline({'text': text, 'text_pair': entailment_text}))

Downloading: 100%|██████████| 1.33k/1.33k [00:00<00:00, 10.5MB/s]


{'label': 'entailment', 'score': 0.9958469271659851}


In [10]:
contradiction_text = '二人の男性が飛んでいます。'
print(nli_pipeline({'text': text, 'text_pair': contradiction_text}))

{'label': 'contradiction', 'score': 0.999154806137085}


In [11]:
neutral_text = '2人の男性が、白い飛行機を眺めています。'
print(nli_pipeline({'text': text, 'text_pair': neutral_text}))

{'label': 'neutral', 'score': 0.9977497458457947}


### 意味的類似度計算

In [12]:
text_sim_pipeline = pipeline(
    model='llm-book/bert-base-japanese-v3-jsts',
    function_to_apply='none'
)
text = '川べりでサーフボードを持った人たちがいます。'
sim_text = 'サーファーたちが川べりに立っています。'
# textとsim_textの類似度を計算
result = text_sim_pipeline({'text': text, 'text_pair': sim_text})
print(result['score'])

Downloading: 100%|██████████| 1.30k/1.30k [00:00<00:00, 10.3MB/s]


3.7349071502685547


In [14]:
dissim_text = 'トイレの壁に黒いタオルがかけられています'
# textとdissim_textの類似度を計算
result = text_sim_pipeline({'text': text, 'text_pair': dissim_text})
print(result['score'])

0.028693273663520813


In [40]:
# 文埋め込みによるコサイン類似度を用いた意味的類似度
import torch
from torch.nn.functional import cosine_similarity

sim_enc_pipeline = pipeline(
    model='llm-book/bert-base-japanese-v3-unsup-simcse-jawiki',
    task='feature-extraction'
)
# textとsim_textのベクトルを獲得
text_emb = torch.tensor(sim_enc_pipeline(text)[0][0])
sim_emb = torch.tensor(sim_enc_pipeline(sim_text)[0][0])
# textとsim_textの類似度を計算
sim_pair_score = cosine_similarity(text_emb, sim_emb, dim=0)
print(sim_pair_score.item())

0.8706234693527222


In [41]:
# dissim_textのエンベディング
dissim_emb = torch.tensor(sim_enc_pipeline(dissim_text)[0][0])
dissim_pair_score = cosine_similarity(text_emb, dissim_emb, dim=0)
print(dissim_pair_score.item())

0.4626588225364685


### 固有表現認識

In [2]:
from pprint import pprint
from transformers import pipeline

ner_pipeline = pipeline(
    model='llm-book/bert-base-japanese-v3-ner-wikipedia-dataset',
    aggregation_strategy='simple',
)
text = '大谷翔平は岩手県水沢市出身のプロ野球選手'
# text中の固有表現を抽出
pprint(ner_pipeline(text))

Downloading: 100%|██████████| 1.45k/1.45k [00:00<00:00, 11.0MB/s]


[{'end': None,
  'entity_group': '人名',
  'score': 0.99823624,
  'start': None,
  'word': '大谷 翔平'},
 {'end': None,
  'entity_group': '地名',
  'score': 0.9986874,
  'start': None,
  'word': '岩手 県 水沢 市'}]


### 要約生成

In [5]:
text2text_pipeline = pipeline(
    model="llm-book/t5-base-long-livedoor-news-corpus"
)
article = "ついに始まった３連休。テレビを見ながら過ごしている人も多いのではないだろうか？　今夜オススメなのは何と言っても、NHKスペシャル「世界を変えた男 スティーブ・ジョブズ」だ。実は知らない人も多いジョブズ氏の養子に出された生い立ちや、アップル社から一時追放されるなどの経験。そして、彼が追い求めた理想の未来とはなんだったのか、ファンならずとも気になる内容になっている。 今年、亡くなったジョブズ氏の伝記は日本でもベストセラーになっている。今後もアップル製品だけでなく、世界でのジョブズ氏の影響は大きいだろうと想像される。ジョブズ氏のことをあまり知らないという人もこの機会にぜひチェックしてみよう。 世界を変えた男　スティーブ・ジョブズ（NHKスペシャル）"
# articleの要約生成
print(text2text_pipeline(article)[0]['generated_text'])

Downloading: 100%|██████████| 1.47k/1.47k [00:00<00:00, 11.6MB/s]


ImportError: /usr/local/lib/python3.10/dist-packages/fused_layer_norm_cuda.cpython-310-x86_64-linux-gnu.so: undefined symbol: _ZN8pybind116detail11type_casterIN3c108ArrayRefIlEEvE4loadENS_6handleEb

### transformersの使い方

In [8]:
from transformers import AutoTokenizer

# AutoTokenizerでトークナイザをロードする
tokenizer = AutoTokenizer.from_pretrained('abeja/gpt2-large-japanese')
# 入力文をトークにする
tokenizer.tokenize('今日は天気が良いので')

['▁', '今日', 'は', '天気', 'が良い', 'の', 'で']

In [9]:
from transformers import AutoModelForCausalLM

# 生成を行うモデルであるAutoModelForCausalLMを使ってモデルをロードする
model = AutoModelForCausalLM.from_pretrained(
    'abeja/gpt2-large-japanese'
)
# トークナイザを使ってモデルへの入力を作成する
inputs = tokenizer('今日は天気が良いので', return_tensors='pt')
# 後続のテキストを予測
outputs = model.generate(
    **inputs,
    max_length=15, # 生成する最大トークン数を15に設定
    pad_token_id=tokenizer.pad_token_id # パディングのトークンidを指定
)
# generate関数の出力をテキスト変換する
generated_text = tokenizer.decode(
    outputs[0], skip_special_tokens=True
)
print(generated_text)

今日は天気が良いので外でお弁当を食べました。
